# Visualization of JEDI analysis with UXarray in the model space

<img src="path or URL to some visual here"
     width="30%"
     alt="MPAS-JEDI visual"
     align="right"
/>

### In this section, you'll learn:

* Utilizing ...

### Related Documentation

* [URL title](URL)
* 

### Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [URL title](URL) | Necessary OR Helpful?  | |

**Time to learn**: 30 minutes?

-----

## Import packages

In [ ]:
%%time 

# autoload external python modules if they changed
%load_ext autoreload
%autoreload 2

# add ../funcs to the current path
import sys, os
sys.path.append(os.path.join(os.getcwd(), "..")) 

# import modules
import warnings
import math

import cartopy.crs as ccrs
import geoviews as gv
import geoviews.feature as gf
import holoviews as hv
import hvplot.xarray
from holoviews import opts
import matplotlib.pyplot as plt

import s3fs

import geopandas as gp
import numpy as np
import uxarray as ux
import xarray as xr

## Configure visualization tools

In [ ]:
# hv.extension("bokeh")
# hv.extension("matplotlib")


# common border lines
coast_lines = gf.coastline(projection=ccrs.PlateCarree(), line_width=1, scale="50m")
state_lines = gf.states(projection=ccrs.PlateCarree(), line_width=1, line_color='gray', scale="50m")

## Load MPAS data
Depending on the network, the data loading process may take a few minutes

### Data paths to the JetStream2 and S3 objects

In [ ]:
%%time

jetstream_url = 'https://js2.jetstream-cloud.org:8001/'
fs = s3fs.S3FileSystem(anon=True, asynchronous=False,client_kwargs=dict(endpoint_url=jetstream_url))
conus12_path = 's3://pythia/mpas/conus12km'

grid_url=f"{conus12_path}/conus12km.invariant.nc_L60_GFS"
bkg_url=f"{conus12_path}/bkg/mpasout.2024-05-06_01.00.00.nc"
ana_url=f"{conus12_path}/ana/mpasout.2024-05-06_01.00.00.nc"

grid_file = fs.open(grid_url)
ana_file = fs.open(ana_url)
bkg_file = fs.open(bkg_url)

### Open UXarray datasets

In [ ]:
%%time 

uxds_a = ux.open_dataset(grid_file, ana_file)
uxds_b = ux.open_dataset(grid_file, bkg_file)

In [ ]:
uxvar = uxds_a['theta'] - 273.15   ## Kelvin to Celsius

In [ ]:
i_lev = 0   # `nVertLevels` index
i_time = 0  # `Time` index

## plot temperature analysis increments at different levels

In [ ]:
%%time

nt=0  # time dimension
plot_levels = [0, 29, 42]  # [0, 29, 42]  # [0, 19, 29, 39, 49, 58]

# Create the colormap
# colors = ['blue', 'white', 'red']    
# cmap = LinearSegmentedColormap.from_list('blue_white_red', colors)
# zero_shift = 0.02 

cmap = plt.get_cmap('coolwarm')
zero_shift = 0.0

plots = []
for lev in plot_levels:
    #  use hslice_contour0() which uses 0 divide the cool and warm colors in the plot by default
    #
    # tmp = hslice_contour0(
    #     uxvar.isel(Time=nt, nVertLevels=lev), 
    #     title=f'lev={lev}', 
    #     cmap=cmap, 
    #     zero_shift=zero_shift, 
    #     clevs_multiplier=1,
    # )  # for the whole domain  

    # hslice_contour() does not dvide the cool and warm colors at 0 by default
    # But it can be achieved by setting symmetric_cmap=True which will set symmetric cmax/cmin automatically,
    #       or mannualy setting symmetric cmax/cmin
    #
    tmp = hslice_contour(
        uxvar.isel(Time=nt, nVertLevels=lev), 
        title=f'lev={lev}',
        symmetric_cmap=True,
        #clevs=20,
    )  # for the whole domain
    
    plots.append(tmp * coast_lines * state_lines)

# plots share one toolbar, which facilitates doing sync'ed zoom-in/out
# hv.Layout(plots).cols(1)

# each plot has its own toolbar, which facilitates controlling each plot individually
for p in plots:
   display(p)

## Zoomed into Colorado using the subset capability

In [ ]:
%%time

lon_center = -105.03
lat_center = 39.0
lon_incr = 5 # degree
lat_incr = 3 # degree
lon_bounds = (lon_center - lon_incr, lon_center + lon_incr)
lat_bounds = (lat_center - lat_incr, lat_center + lat_incr)

### subset to a small domain
uxdiff1 = uxdiff0.subset.bounding_box(lon_bounds, lat_bounds,)
uxvar = uxdiff1


nt=0  # time dimension
plot_levels = [42]  # [0, 29, 42]  # [0, 19, 29, 39, 49, 58]

plots = []
for lev in plot_levels:
    tmp = hslice_contour(uxvar.isel(Time=nt, nVertLevels=lev), title=f'lev={lev}', width=700, height=500)  # for the subdomain
    
    # overlay state_lines
    #plots.append(tmp * coast_lines * state_lines)  
    
    # overlay county lines, this takes longer time to render
    plots.append(tmp * coast_lines.opts(xlim=(lon_bounds[0], lon_bounds[1]), ylim=(lat_bounds[0], lat_bounds[1])))

# plots share one toolbar, which facilitates doing sync'ed zoom-in/out
# hv.Layout(plots).cols(1)

# each plot has its own toolbar, which facilitates controlling each plot individually
for p in plots:
   display(p)

## vertical cross section of temperature increments

In [ ]:
%%time

# tmp = vslice_contour(uxvar, lon=-85.77, clevels=10)
# display(tmp)
tmp = vslice_contour(uxvar, lat=42.63, clevels=10)
display(tmp)

## save plots to files

In [ ]:
hv.save(tmp, 'vslice.png')